# 2.4.1 종목 코드 추출하기

In [2]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np

# 거래소 전체 추출 (코스피, 코스닥 대상) 
df = fdr.StockListing('KRX').query("Market in ['KOSPI', 'KOSDAQ']")

# 종목별 종가(Close) 문자형에서 수치형(integer)으로 변환
df['Close'] = df['Close'].astype(int) 

In [4]:
# 소형주: 시가총액 1,000억 원 미만 종목 제외 적용  
df = df.query(" Marcap >= 100000000000 ")

# 저유동성 : 일 거래량 2,000주 미만 제외 적용 
df = df.query(" Volume >= 2000 ")

# 동전주: 높은 변동성 위험을 고려하여 주가 1,000원 미만 종목 제외 적용 
df = df.query(" Close >= 1000 ")

# 우선주: 낮은 거래량과 본주와의 주가 차이(괴리율)로 인한 분석의 한계로 제외 (종목코드 끝자리 5,7,9,K) 적용 
df = df[ df['Code'].apply(lambda x: False if x[-1] in ['5','7','9','K'] else True) ]

# 스팩(SPAC), 리츠, 벤처투자 등 : 재무 데이터의 실효성이 없어 제외 적용 
df = df[ ~df['Name'].str.contains('스팩|리츠|리얼티|인프라|유전|벤처') ]

# 데이터프레임 인덱스 초기화 
df = df.reset_index(drop=True) 

# 최종 종목 추출 건수 출력 
print(len(df)) 


1328


In [5]:
# 종목 정보 사전 저장 
stock_list = dict(zip(df['Code'], df['Name']))

# (참고) 수집 결과 출력 (최초 5건)
print("분석대상 기업 건수 ", len(stock_list))
print("최초 5건 출력 : ")
for i, (k, v) in enumerate(stock_list.items()):
    if i <= 4:
        print(k, v)

분석대상 기업 건수  1328
최초 5건 출력 : 
005930 삼성전자
000660 SK하이닉스
207940 삼성바이오로직스
373220 LG에너지솔루션
005380 현대차


In [ ]:
# 파일 저장에 필요한 라이브러리 호출
import pickle

# 1. 파일 저장
with open('멀티팩터_전체종목_stock_list.pkl', 'wb') as f:
    pickle.dump(stock_list, f)

# 2. 파일 불러오기
with open('멀티팩터_전체종목_stock_list.pkl', 'rb') as f:
    stock_list = pickle.load(f)


# 2.4.2 종합지표 생성하기

In [6]:
def get_momentum(scode, SD, ED):
    """    
    주가 모멘텀과 거래량 모멘텀 계산 

    매개변수: 
         scode : 종목 코드 (예: '005930')
         SD : 주가데이터 추출시작일 (예: '20250102')
         ED : 주가데이터 추출종료일 (예: '20250131')

    반환값: 
        avg : 주가 모멘텀 지표 (기간별 수익률 평균)
        vol : 거래량 모멘텀 지표 
    """

    # 1. 주가 및 거래량 데이터 수집 (여기는 에러나면 멈춰서 확인해야 하므로 try-except 제거)
    df = fdr.DataReader(scode, SD, ED)

    # 2. 주가 모멘텀 지표 생성
    # 단기 (1일, 1주, 1개월) - 데이터가 너무 없으면 에러가 나는 게 맞으므로 그대로 둠
    mom_1d = ( df.iloc[-1]['Close'] / df.iloc[-2]['Close'] - 1 )  * 100    # 전일
    mom_1w = ( df.iloc[-1]['Close'] / df.iloc[-5]['Close'] - 1 )  * 100    # 주간
    mom_1m = ( df.iloc[-1]['Close'] / df.iloc[-20]['Close'] - 1 )  * 100   # 월간 

    # 3개월간 (데이터 부족 시 NaN 처리)
    try:
        mom_3m = ( df.iloc[-1]['Close'] / df.iloc[-60]['Close'] - 1 )  * 100 
    except:
        mom_3m = np.nan

    # 연간 (데이터 부족 시 NaN 처리)
    try:
        mom_1y = ( df.iloc[-1]['Close'] / df.iloc[-240]['Close'] - 1 )  * 100
    except:
        mom_1y = np.nan

    # 3. 기간별 수익률의 평균 계산 (NaN을 제외하고 평균 산출)
    avg = np.nanmean([mom_1d, mom_1w, mom_1m, mom_3m, mom_1y])

    # 4. 거래량 모멘텀 지표 생성
    try:
        vol_short = df['Volume'].rolling(window=5).mean().iloc[-1]  # 최근 5일
        vol_long = df['Volume'].rolling(window=20).mean().iloc[-1]  # 최근 20일
        vol = ( vol_short / vol_long - 1) * 100
    except:
        vol = np.nan

    return avg, vol


# 모멘텀 지표 계산 후 result에 적재  
result = []
for scode, sname in stock_list.items():
    try: 
        mom, vol = get_momentum(scode, '20240930', '20250930')  # 최근 1년
        result.append( [ scode, sname, mom, vol ] )

    except: 
        print("추출오류 : ", sname) # 에러 종목코드 출력
        result.append([scode, sname, None, None])

# 모멘텀 지표 결과(result)는 데이터프레임(data_mom)으로 저장
data_mom = pd.DataFrame(result, columns=['scode', 'sname', 'mom_price', 'mom_vol'])

# 추출 오류 종목 제외 (최근 상장 종목) 
data_mom = data_mom.dropna().reset_index(drop=True)

# 마지막 5건 출력
data_mom.tail()


추출오류 :  삼성에피스홀딩스
추출오류 :  에임드바이오
추출오류 :  알지노믹스
추출오류 :  리브스메드
추출오류 :  노타
추출오류 :  명인제약
추출오류 :  세미파이브
추출오류 :  씨엠티엑스
추출오류 :  비츠로넥스텍
추출오류 :  삼양바이오팜
추출오류 :  큐리오시스
추출오류 :  더핑크퐁컴퍼니
추출오류 :  테라뷰
추출오류 :  나라스페이스테크놀로지
추출오류 :  티엠씨
추출오류 :  에스엔시스
추출오류 :  에스투더블유
추출오류 :  그린광학
추출오류 :  세나테크놀로지
추출오류 :  아크릴
추출오류 :  이노테크
추출오류 :  삼익제약
추출오류 :  페스카로
추출오류 :  쿼드메디슨
추출오류 :  아로마티카
추출오류 :  삼진식품
추출오류 :  이지스


,scode,sname,mom_price,mom_vol
1296,102370,케이옥션,-3.811211,-38.856348
1297,067290,JW신약,-4.055247,-15.384405
1298,206560,덱스터,-13.475961,26.912308
1299,079810,디이엔티,-10.841780,4.009355
1300,071670,에이테크솔루션,-5.465417,-6.586890


In [7]:
import time 

result = []

for i, (scode, sname) in enumerate(stock_list.items()):

    # [중요] 데이터 수집 시 서버 차단 방지 (1초 대기)
    time.sleep(1)

    # 진행 상황 모니터링 (50번째 순서마다 종목명 출력)
    if i % 50 == 0:
        print(f"{i}/{len(stock_list)} 진행 중... ({sname})")

    try: 
        # 재무제표 데이터 추출 (연간 K-IFRS 연결 기준)
        data = fdr.SnapDataReader(f'NAVER/FINSTATE-2Y/{scode}').dropna(subset='영업이익') # 연간
        per, pbr = data['PER(배)'].iloc[-1], data['PBR(배)'].iloc[-1]
        result.append([scode, per, pbr])

    except: 
        print("추출오류 : ", sname) # 에러 종목코드 출력
        result.append([scode, None, None])

0/1328 진행 중... (삼성전자)
추출오류 :  삼성에피스홀딩스
50/1328 진행 중... (KT)
추출오류 :  카카오뱅크
추출오류 :  펩트론
추출오류 :  코오롱티슈진
100/1328 진행 중... (JB금융지주)
추출오류 :  산일전기
추출오류 :  에코프로머티
추출오류 :  알지노믹스
150/1328 진행 중... (미스토홀딩스)
추출오류 :  시프트업
200/1328 진행 중... (이수스페셜티케미컬)
추출오류 :  이수스페셜티케미컬
추출오류 :  씨어스테크놀로지
추출오류 :  코스모신소재
추출오류 :  지투지바이오
추출오류 :  에이프릴바이오
250/1328 진행 중... (루닛)
추출오류 :  인벤티지랩
추출오류 :  프로티나
추출오류 :  앱클론
추출오류 :  지아이이노베이션
추출오류 :  온코닉테라퓨틱스
추출오류 :  인투셀
300/1328 진행 중... (명인제약)
추출오류 :  이뮨온시아
추출오류 :  케이씨텍
추출오류 :  에이치브이엠
추출오류 :  파미셀
추출오류 :  케이카
추출오류 :  넥스트바이오메디컬
350/1328 진행 중... (하나투어)
추출오류 :  아이쓰리시스템
추출오류 :  동운아나텍
추출오류 :  삼양컴텍
추출오류 :  제룡전기
400/1328 진행 중... (유진로봇)
추출오류 :  HLB제약
추출오류 :  태웅
추출오류 :  비츠로넥스텍
추출오류 :  큐렉소
추출오류 :  롯데손해보험
추출오류 :  쓰리빌리언
450/1328 진행 중... (OCI)
추출오류 :  안트로젠
추출오류 :  일진하이솔루스
추출오류 :  PI첨단소재
추출오류 :  조광피혁
추출오류 :  나이벡
추출오류 :  엔젤로보틱스
추출오류 :  기가비스
500/1328 진행 중... (파인엠텍)
추출오류 :  와이바이오로직스
추출오류 :  동국제강
추출오류 :  삼양바이오팜
추출오류 :  천일고속
추출오류 :  바이오다인
추출오류 :  샘씨엔에스
550/1328 진행 중... (부광약품)
추출오류 :  나라스페이스테크놀로지
추출오류 :  

NameError: name 'result_val' is not defined

In [8]:
data_val = pd.DataFrame(result, columns=['scode', 'PER', 'PBR'])

data_val.head()

,scode,PER,PBR
0,005930,10.49,0.89
1,000660,6.39,1.62
2,207940,91.76,9.12
3,373220,240.49,2.63
4,005380,4.36,0.47


In [ ]:
import time 

result = []
for i, (scode, sname) in enumerate(stock_list.items()):

    # [중요] 1. 서버 차단 방지 (1초 대기)
    time.sleep(1)

    # 진행 상황 모니터링 (50개마다 출력)
    if i % 50 == 0:
        print(f"{i}/{len(stock_list)} 진행 중... ({sname})")

    try: 
        # 재무 데이터 추출 : 분기 K-IFRS 연결 기준 
        data = fdr.SnapDataReader(f'NAVER/FINSTATE-2Q/{scode}').dropna(subset='영업이익')  # 분기
        fin = (data[['매출액','영업이익','당기순이익']].pct_change()*100).iloc[-1].to_list()
        roe = data['ROE(%)'].iloc[-1]
        result.append([scode] + fin + [roe])

    except: 
        print("추출오류 : ", sname) # 에러 종목코드 출력
        result.append([scode, None, None, None, None])


# ROE와 나머지 지표를 결합하여, 퀄러티 지표 완성 
data_fin = pd.DataFrame(result_fin, columns=['scode', 'revenue_rate', 'oper_income_rate', 'net_income_rate', 'ROE'])

data_fin.head()  


0/1328 진행 중... (삼성전자)
추출오류 :  삼성에피스홀딩스
50/1328 진행 중... (KT)
추출오류 :  카카오뱅크
추출오류 :  펩트론
추출오류 :  코오롱티슈진
100/1328 진행 중... (JB금융지주)
추출오류 :  한화엔진
추출오류 :  에코프로머티
추출오류 :  셀트리온제약


In [ ]:
data_mom.to_pickle("멀티팩터_전체종목_모멘텀.pkl")
data_val.to_pickle("멀티팩터_전체종목_밸류.pkl")
data_fin.to_pickle("멀티팩터_전체종목_퀄러티.pkl")

In [ ]:
# 멀티팩터 마스터 파일 생성 
data_mast = data_mom.merge(data_val, on='scode', how='left')\
                    .merge(data_fin, on='scode', how='left')

# 멀티팩터 마스터파일에서 수치형 컬럼명을 추출하여 cols로 저장
cols = data_mast.select_dtypes(exclude='object').columns

# for문으로 cols에 저장된 컬럼명(멀티팩터)별로 반복. 결측치 발생시 중앙값으로 대체
for col in cols:
    data_mast[col] = data_mast[col].replace([np.inf, -np.inf], np.nan) # 무한대 처리 
    data_mast[col] = data_mast[col].fillna(data_mast[col].median())    # 결측치 처리


In [ ]:
# 모멘텀 지표 순위화 : 값이 클수록 우수 
data_mast['모멘텀_주가'] = data_mast['mom_price'].rank(ascending=False, axis=0) / len(data_mast) * 100  
data_mast['모멘텀_거래량'] = data_mast['mom_vol'].rank(ascending=False, axis=0) / len(data_mast) * 100  

# 밸류 지표 순위화 : 값이 작을수록 우수
data_mast['밸류_PER'] = data_mast['PER'].rank(ascending=True, axis=0) / len(data_mast) * 100   
data_mast['밸류_PBR'] = data_mast['PBR'].rank(ascending=True, axis=0) / len(data_mast) * 100   
data_mast['퀄러티_ROE'] = data_mast['ROE'].rank(ascending=False, axis=0) / len(data_mast) * 100 

# 퀄러티 지표 순위화 : 값이 클수록 우수 
data_mast['퀄러티_ROE'] = data_mast['ROE'].rank(ascending=False, axis=0) / len(data_mast) * 100
data_mast['퀄러티_매출증가'] = data_mast['revenue_rate'].rank(ascending=False, axis=0) / len(data_mast) * 100   
data_mast['퀄러티_영업이익증가'] = data_mast['oper_income_rate'].rank(ascending=False, axis=0) / len(data_mast) * 100   
data_mast['퀄러티_순이익증가'] = data_mast['net_income_rate'].rank(ascending=False, axis=0) / len(data_mast) * 100   

# 멀티팩터 종합점수 계산 컬럼 정의
cols = ['모멘텀_주가', '모멘텀_거래량', '밸류_PER', '밸류_PBR', '퀄러티_ROE', '퀄러티_매출증가', '퀄러티_영업이익증가', '퀄러티_순이익증가']

# 종합점수 계산 (평균) 
data_mast['종합점수'] = np.average( data_mast[cols], axis=1 ) 

# 종합순위 계산 
data_mast['종합순위'] = data_mast['종합점수'].rank(ascending=True, axis=0)

# 종합순위 퍼센트 계산 
data_mast['종합순위_퍼센트'] = data_mast['종합순위'] / len(data_mast) * 100  

# 종합순위 상위 순으로 정렬
data_mast.sort_values(by='종합순위', inplace=True)

df_mast.head(10).T


In [ ]:
# 피클 파일 저장 
df_mast.to_pickle('df_mast_all.pkl')

# (참고) 피클 파일 불러오기 
df_mast = pd.read_pickle('df_mast_all.pkl')

# 2.4.3 종합지표 분석하기

In [ ]:
#  분석 대상 종목 주가 등락률 추출  
stocks = ','.join( stock_list.keys() )
data = fdr.DataReader(stocks, '20251001', '20251230')   

first_row = data.iloc[0]  # 10월 1일
last_row = data.iloc[-1]  # 12월 30일
ret = ((last_row - first_row) / first_row) * 100

df = pd.DataFrame({'scode':ret.index, 'RET':ret.values})

# 최종 3건 화면 출력 
df.tail(3)

In [ ]:
# 멀티팩터 테이블과 주가 데이터 결합 
data_ret = data_mast.merge(df, on='scode', how='left')

# 최종 5건 화면 출력 (T옵션으로 행과 열을 교환하여 출력) 
data_ret.head(5).T

In [ ]:
data_ret.to_pickle('멀티팩터_전체종목_mast_검증용주가포함.pkl')

In [ ]:
import seaborn as sns 
sns.histplot(x='RET', data=data_ret)
plt.xlabel('주가등락률(25년4분기)')

In [ ]:
data_ret['RET'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])

In [ ]:
data_ret.query("RET < 41 & RET > -17 ")

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc
import platform

# 1. 한글 폰트 설정
if platform.system() == 'Windows':
    rc('font', family='Malgun Gothic')  # 윈도우: 맑은 고딕
elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic')    # 맥: 애플고딕
else:
    rc('font', family='NanumBarunGothic') # 리눅스(코랩 등)

# 2. 마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False
def plot_by_factor(df, feat):
    ser = pd.qcut(df[feat], 10, duplicates='drop')  # feat 컬럼별 10개 그룹화 
    print( df.groupby(ser)['RET'].agg(['size', 'mean', 'median']) )  # 그룹별 건수, 평균, 중앙값 출력 
    df.groupby(ser)['RET'].mean().plot(kind='bar') # 그룹별 수익률 평균으로 막대 그래프 생성
    plt.show()
def plot_by_factor(df, feat):
    ser = pd.qcut(df[feat], 10, duplicates='drop')  # feat 컬럼별 10개 그룹화 
    print( df.groupby(ser)['RET'].agg(['size', 'mean', 'median']) )  # 그룹별 건수, 평균, 중앙값 출력 
    df.groupby(ser)['RET'].mean().plot(kind='bar') # 그룹별 수익률 평균으로 막대 그래프 생성
    plt.show()


In [ ]:
plot_by_factor(data_ret.query("RET < 41 & RET > -17 "), '종합순위')

In [ ]:
plot_by_factor(data_ret.query("RET < 41 & RET > -17 "), '모멘텀_주가')

In [ ]:
plot_by_factor(data_ret.query("RET < 41 & RET > -17 "), '모멘텀_거래량')

In [ ]:
plot_by_factor(data_ret.query("RET < 41 & RET > -17 "), '밸류_PER')

In [ ]:
plot_by_factor(data_ret.query("RET < 41 & RET > -17 "), '밸류_PBR')

In [ ]:
plot_by_factor(data_ret.query("RET < 41 & RET > -17 "), '퀄러티_ROE')

In [ ]:
plot_by_factor(data_ret.query("RET < 41 & RET > -17 "), '퀄러티_매출증가')

In [ ]:
plot_by_factor(data_ret.query("RET < 41 & RET > -17 "), '퀄러티_영업이익증가')

In [ ]:
plot_by_factor(data_ret.query("RET < 41 & RET > -17 "), '퀄러티_순이익증가')

In [ ]:
ed!# 주가(RET)와 멀티팩터 지표의 상관계수 행렬 생성 
cor = ( data_ret.query("RET < 41 & RET > -17 ")[['RET', '모멘텀_주가', '모멘텀_거래량', '밸류_PER', '밸류_PBR', '퀄러티_ROE', '퀄러티_매출증가', '퀄러티_영업이익증가', '퀄러티_순이익증가']]
                .corr() )

# 상관계수 행렬에서 주가(RET)만 추출 > 상관계수 크기로 정렬 > 막대그래프 생성 
cor['RET'].iloc[1:].sort_values().plot(kind='barh')


# 2.4.4 종합지표 최적화하기

In [ ]:
# 멀티팩터 지표 컬럼명 정의 
cols = ['모멘텀_주가', '모멘텀_거래량', '밸류_PER', '밸류_PBR', '퀄러티_ROE', '퀄러티_매출증가', '퀄러티_영업이익증가', '퀄러티_순이익증가']

# 가중치 설정 데이터프레임 정의 
df_w = pd.DataFrame()
df_w['cols'] = cols
df_w['weights'] = [0/8, 0/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8]  

# 가중치 데이터프레임 화면 출력 
df_w


In [ ]:
# 가중치 적용을 위한 멀티팩터 마스터(data_mast)의 복사본(dat_weight) 생성
data_weight = df_mast.copy() 

# 가중치를 적용하여 종합점수 생성 
data_weight['종합점수'] = np.average( data_weight[cols], axis=1, weights=df_w['weights']) 

# 종합순위 및 종합순위 퍼센트 생성 
data_weight['종합순위'] = data_weight['종합점수'].rank(ascending=True, axis=0)
data_weight['종합순위_퍼센트'] = data_weight['종합순위'] / len(data_weight) * 100 

# 종합순위 순으로 정렬
data_weight.sort_values(by='종합순위', inplace=True)

# 종합순위 상위 10개 종목 화면 출력
data_weight.head(10).T


In [ ]:
# 멀티팩터 지표 컬럼명 정의 
df_w = pd.DataFrame()

# 가중치 설정 데이터프레임 정의 
df_w['cols'] = cols
df_w['weights'] = [1/8, 1/8, 0/8, 0/8, 1/8, 1/8, 1/8, 1/8] 

# 가중치 데이터프레임 화면 출력 
df_w 


In [ ]:
# 가중치 적용을 위한 멀티팩터 마스터(data_mast)의 복사본(dat_weight) 생성  
data_weight = data_mast.copy() 

# 가중치를 적용하여 종합점수 생성 
data_weight['종합점수'] = np.average( data_weight[cols], axis=1, weights=df_w['weights']) 

# 종합순위 및 종합순위 퍼센트 생성 
data_weight['종합순위'] = data_weight['종합점수'].rank(ascending=True, axis=0)
data_weight['종합순위_퍼센트'] = data_weight['종합순위'] / len(data_weight) * 100

# 종합순위 순으로 정렬
data_weight.sort_values(by='종합순위', inplace=True)

# 종합순위 상위 10개 종목 화면 출력
data_weight.head(10).T


In [ ]:
# 멀티팩터 지표 컬럼명 정의 
cols = ['모멘텀_주가', '모멘텀_거래량', '밸류_PER', '밸류_PBR', '퀄러티_ROE', '퀄러티_매출증가', '퀄러티_영업이익증가', '퀄러티_순이익증가']

# 가중치 설정 데이터프레임 정의 
df_w = pd.DataFrame()
df_w['cols'] = cols
df_w['weights'] = [1/8, 1/8, 1/8, 1/8, 0/8, 0/8, 0/8, 0/8] 

# 가중치 데이터프레임 화면 출력 
df_w 


In [ ]:
# 가중치 적용을 위한 멀티팩터 마스터(data_mast)의 복사본(dat_weight) 생성  
data_weight = data_mast.copy() 

# 가중치를 적용하여 종합점수 생성 
data_weight['종합점수'] = np.average( data_weight[cols], axis=1, weights=df_w['weights']) 

# 종합순위 및 종합순위 퍼센트 생성 
data_weight['종합순위'] = data_weight['종합점수'].rank(ascending=True, axis=0)
data_weight['종합순위_퍼센트'] = data_weight['종합순위'] / len(data_weight) * 100

# 종합순위 순으로 정렬
data_weight.sort_values(by='종합순위', inplace=True)

# 종합순위 상위 10개 종목 화면 출력
data_weight.head(10).T
